# Intrinsics

## Indexing

### Subscript

A subscript is a mapping from the dimensions (axes of the shape) to a collection of indexes along this axis.

Let's say $n_d$ is the number of dimensions of an array $A$, and $d_0, d_1, ... d_{n_d}$ are the sizes along those dimensions. We can further define $F$, a filter as a sequence $(f_0, f_1, ... f_{n_d})$ where for any $i$, $f_i$ is the sequence of selected indexes along this axis. 

If $(i_0, i_1, ..., i_{n_d})$ are the indexes of an element in $A$ and $D$ is the (flat) buffer storing this data, then:

$$A(i_0, i_1, ..., i_{n_d}) = D(\sum_{k=0}^{n_d} i_k s_k)$$

where the $s_i$ are the strides of $A$.

$$\forall i \in \{0, 1, ..., n_d - 1\}, s_i = \prod_{k=i+1}^{n_d} d_k \text{ and } s_{n_d} = 1$$

If we have $I = \sum_{k=0}^{n_d} i_k s_k$ then if we want to find $I'$ for $i_{k_{new}}$ for example: we can do $I' = I + s_k(i_{k_{new}} - i_k)$


## Matmul

#### Shape compatibility

$(...,a, b) \times (..., b, c) \rightarrow (...,a, c)$

not implemented $(...,a) \times (..., 1, b) \rightarrow (...,a, b)$

not implemented $(...,a, b) \times (..., b) \rightarrow (...,a)$



## Derivatives

$f(u) = ln(u)$

$f'(u) = \frac{1}{u}$

$f(v) = softmax(v) = \frac{e^v}{reduce\_sum(e^v)} = \frac{e^v}{\sum e^{v_i}}$
$f(v)_i = softmax(v)_i = \frac{e^{v_i}}{\sum_j e^{v_j}}$

$f'(v)_i = \frac{e^{v_i} \sum_j e^{v_j} - e^{2v_i}}{(\sum_j e^{v_j})^2} = softmax(v)_i - softmax(v)_i^2$

$f'(v) = softmax(v) - softmax(v)^2$



$L(y, u) = -\sum y_i$